# Patch Classifier Query
We are now running both a pixel and patch classifier on Descartes. This notebook takes in a set of patch classifier outputs along with a set of pixel classifier candidates. This notebook finds the intersection between candidate points and patch classifier predictions above a given threshold.

In [ ]:
import geopandas as gpd
# pygeos is required to use geopandas spatial indexing
gpd.options.use_pygeos = True

In [ ]:
region_name = 'brazil'

In [ ]:
patch_model_version = '3.0'
patch_classifier_fname= f'{region_name}_patch_weak_labels_3.0_2019-01-01_2021-06-01_stride_8'
patch = gpd.read_file(f'../data/model_outputs/patch_classifier/{patch_classifier_fname}.geojson')

In [ ]:
pixel_model_version = '0.0.11'
threshold = 0.6
min_sigma = 3.5
pixel_classifier_fname = f'{region_name}_v0.0.11_2019-01-01_2021-06-01_blobs_thresh_{threshold}_min-sigma_{min_sigma}_area-thresh_0.0025'
pixel = gpd.read_file(f'../data/model_outputs/candidate_sites/{pixel_model_version}/{pixel_classifier_fname}.geojson')

In [ ]:
threshold = 0.3
patch_threshold = patch[patch['mean'] > threshold]
patch_index = patch_threshold['geometry'].sindex

In [ ]:
overlap = []
for candidate in pixel['geometry']:
    if len(patch_index.query(candidate)) > 0:
        overlap.append(True)
    else:
        overlap.append(False)
union = pixel[overlap]
print(f"{len(union)} ({len(union) / len(pixel):.0%}) candidate points intersect with patch classifier predictions greater than {threshold}")
union

In [ ]:
file_path = f'../data/model_outputs/candidate_sites/{pixel_model_version}/{pixel_classifier_fname}_patch_v{patch_model_version}_threshold_{threshold}_intersection.geojson'
union.to_file(file_path, driver='GeoJSON')
print("Saved to", file_path)